In [6]:
import os
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_core.messages import ToolMessage
from langchain_groq import ChatGroq

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
api_key = os.getenv("GROQ_API_KEY")


model = ChatGroq(model_name="openai/gpt-oss-120b", api_key=api_key)
model.invoke("test")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'reasoning_content': 'We have a conversation: system prompt defines behavior. The user says "test". The developer message says nothing special. So we should respond normally. Probably just respond "Hello! How can I help you today?" or something. Probably a simple reply.'}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 72, 'total_tokens': 141, 'completion_time': 0.14262599, 'completion_tokens_details': {'reasoning_tokens': 51}, 'prompt_time': 0.002752389, 'prompt_tokens_details': None, 'queue_time': 0.055263561, 'total_time': 0.145378379}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_d29d1d1418', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b727f-5017-7533-afb0-83fbfdf37ef7-0', usage_metadata={'input_tokens': 72, 'output_tokens': 69, 'total_tokens': 141, 'output_token_details': {'reasoning': 51}})

In [12]:
@tool
def suhu_ruangan(ruangan:float)->float:
    """ ini adalah fungsi untuk mencari suhu ruangan """
    return f"Suhu {ruangan} adalah 54 Celcius"

model_with_tools = model.bind_tools([suhu_ruangan])

In [14]:
## step User Ask
query = "Berapa kah suhu kamar saat ini?"
messages = [{
    "role": "user",
    "content": query
}]




In [16]:
## Mengecek AI Perlu alat atau tidak

ai_msg = model_with_tools.invoke(messages)

print(ai_msg)

content='' additional_kwargs={'reasoning_content': 'The user asks: "Berapa kah suhu kamar saat ini?" They want current room temperature. We have a function "suhu_ruangan" that takes "ruangan: number". Probably we need to call it with a room identifier. Not sure what number to use. Could be 1 as default. We\'ll call the function.', 'tool_calls': [{'id': 'fc_d3020943-430e-4ebd-8c5e-26648ecfc3a8', 'function': {'arguments': '{"ruangan":1}', 'name': 'suhu_ruangan'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 132, 'total_tokens': 230, 'completion_time': 0.202990802, 'completion_tokens_details': {'reasoning_tokens': 68}, 'prompt_time': 0.005988727, 'prompt_tokens_details': None, 'queue_time': 0.054402472, 'total_time': 0.208979529}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b7283-a2d9-7b

In [21]:
ai_msg.tool_calls

[{'name': 'suhu_ruangan',
  'args': {'ruangan': 1},
  'id': 'fc_d3020943-430e-4ebd-8c5e-26648ecfc3a8',
  'type': 'tool_call'}]

In [22]:
messages.append(ai_msg)

In [23]:
messages

[{'role': 'user', 'content': 'Berapa kah suhu kamar saat ini?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'The user asks: "Berapa kah suhu kamar saat ini?" They want current room temperature. We have a function "suhu_ruangan" that takes "ruangan: number". Probably we need to call it with a room identifier. Not sure what number to use. Could be 1 as default. We\'ll call the function.', 'tool_calls': [{'id': 'fc_d3020943-430e-4ebd-8c5e-26648ecfc3a8', 'function': {'arguments': '{"ruangan":1}', 'name': 'suhu_ruangan'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 132, 'total_tokens': 230, 'completion_time': 0.202990802, 'completion_tokens_details': {'reasoning_tokens': 68}, 'prompt_time': 0.005988727, 'prompt_tokens_details': None, 'queue_time': 0.054402472, 'total_time': 0.208979529}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'service_tier': 'on_demand', 'finish_reason': 'tool_ca

In [27]:
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        if tool_call['name'] == suhu_ruangan:
            suhu = suhu_ruangan.invoke(tool_call)
            messages.append(suhu)

    final_response = model_with_tools.invoke(messages)
    print(final_response)


content='Saat ini suhu kamar (ruangan\u202f1) adalah **54\u202f°C**.' additional_kwargs={'reasoning_content': 'The user asks "Berapa kah suhu kamar saat ini?" The function suhu_ruangan expects a ruangan number. We called with ruangan 1. The function returned "Suhu 1.0 adalah 54 Celcius". Probably we need to present answer.'} response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 173, 'total_tokens': 254, 'completion_time': 0.169628603, 'completion_tokens_details': {'reasoning_tokens': 55}, 'prompt_time': 0.006542431, 'prompt_tokens_details': None, 'queue_time': 0.054747559, 'total_time': 0.176171034}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_4a19b1544c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b7289-4eb1-74b1-930c-535608ef4678-0' usage_metadata={'input_tokens': 173, 'output_tokens': 81, 'total_tokens': 254, 'output_token_details': {'reasoning': 55}}


In [46]:
@tool
def dapatkan_cuaca(kota:str) -> str:
    """ ini adalah tool untuk mendapatkan suhu suatu kota """
    return f"Suhu {kota} adalah 20 derajat celsius"

model_with_tools = model.bind_tools([dapatkan_cuaca, suhu_ruangan])

In [ ]:

query = [{
    "role" : "user",
    "content" : "berapa suhu kota BANDUNG saat ini?"
}]

respon = model_with_tools.invoke(query)
respon

AIMessage(content='', additional_kwargs={'reasoning_content': 'The user asks for current temperature of Bandung. Use function dapatkan_cuaca with kota "Bandung".', 'tool_calls': [{'id': 'fc_01c2b994-0d74-4738-8ef0-e3c7fb46221a', 'function': {'arguments': '{"kota":"Bandung"}', 'name': 'dapatkan_cuaca'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 160, 'total_tokens': 216, 'completion_time': 0.118074461, 'completion_tokens_details': {'reasoning_tokens': 23}, 'prompt_time': 0.006770083, 'prompt_tokens_details': None, 'queue_time': 0.055656357, 'total_time': 0.124844544}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7292-5bcd-7b01-a86c-d45efff5df27-0', tool_calls=[{'name': 'dapatkan_cuaca', 'args': {'kota': 'Bandung'}, 'id': 'fc_01c2b994-0d74-4738-8ef0-e3c7fb46221a', 'type': 'tool_call

In [48]:
query.append(respon)
query = query[0 : 2]
query

[{'role': 'user', 'content': 'berapa suhu kota BANDUNG saat ini?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'The user asks for current temperature of Bandung. Use function dapatkan_cuaca with kota "Bandung".', 'tool_calls': [{'id': 'fc_01c2b994-0d74-4738-8ef0-e3c7fb46221a', 'function': {'arguments': '{"kota":"Bandung"}', 'name': 'dapatkan_cuaca'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 160, 'total_tokens': 216, 'completion_time': 0.118074461, 'completion_tokens_details': {'reasoning_tokens': 23}, 'prompt_time': 0.006770083, 'prompt_tokens_details': None, 'queue_time': 0.055656357, 'total_time': 0.124844544}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7292-5bcd-7b01-a86c-d45efff5df27-0', tool_calls=[{'name': 'dapatkan_cuaca', 'args': {'kota': 'Bandung'}

In [49]:
respon.tool_calls

[{'name': 'dapatkan_cuaca',
  'args': {'kota': 'Bandung'},
  'id': 'fc_01c2b994-0d74-4738-8ef0-e3c7fb46221a',
  'type': 'tool_call'}]

In [52]:
if respon.tool_calls:
    for tool_call in respon.tool_calls:
        if tool_call['name'] == "dapatkan_cuaca":
            hasil_suhu = dapatkan_cuaca.invoke(tool_call)
            query.append(hasil_suhu)

    final_response = model_with_tools.invoke(query)
    print(final_response)

content='Suhu saat ini di kota Bandung adalah **20\u202f°C**.' additional_kwargs={'reasoning_content': 'The user asks: "berapa suhu kota BANDUNG saat ini?" The assistant already called the function and got response: "Suhu Bandung adalah 20 derajat celsius". We should respond in Indonesian, providing the temperature. Possibly add context.'} response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 205, 'total_tokens': 278, 'completion_time': 0.151244881, 'completion_tokens_details': {'reasoning_tokens': 50}, 'prompt_time': 0.008241843, 'prompt_tokens_details': None, 'queue_time': 0.055359697, 'total_time': 0.159486724}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b7294-edb2-7392-9038-120175fc60ce-0' usage_metadata={'input_tokens': 205, 'output_tokens': 73, 'total_tokens': 278, 'output_token_details': {'reasoning': 50}}


In [53]:
final_response

AIMessage(content='Suhu saat ini di kota Bandung adalah **20\u202f°C**.', additional_kwargs={'reasoning_content': 'The user asks: "berapa suhu kota BANDUNG saat ini?" The assistant already called the function and got response: "Suhu Bandung adalah 20 derajat celsius". We should respond in Indonesian, providing the temperature. Possibly add context.'}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 205, 'total_tokens': 278, 'completion_time': 0.151244881, 'completion_tokens_details': {'reasoning_tokens': 50}, 'prompt_time': 0.008241843, 'prompt_tokens_details': None, 'queue_time': 0.055359697, 'total_time': 0.159486724}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7294-edb2-7392-9038-120175fc60ce-0', usage_metadata={'input_tokens': 205, 'output_tokens': 73, 'total_tokens': 278, 'output_token_details': {'reasoning

In [54]:
query

[{'role': 'user', 'content': 'berapa suhu kota BANDUNG saat ini?'},
 AIMessage(content='', additional_kwargs={'reasoning_content': 'The user asks for current temperature of Bandung. Use function dapatkan_cuaca with kota "Bandung".', 'tool_calls': [{'id': 'fc_01c2b994-0d74-4738-8ef0-e3c7fb46221a', 'function': {'arguments': '{"kota":"Bandung"}', 'name': 'dapatkan_cuaca'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 160, 'total_tokens': 216, 'completion_time': 0.118074461, 'completion_tokens_details': {'reasoning_tokens': 23}, 'prompt_time': 0.006770083, 'prompt_tokens_details': None, 'queue_time': 0.055656357, 'total_time': 0.124844544}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_8a618bed98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7292-5bcd-7b01-a86c-d45efff5df27-0', tool_calls=[{'name': 'dapatkan_cuaca', 'args': {'kota': 'Bandung'}